In [442]:
import os

In [443]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'

In [444]:
os.chdir("../")

In [445]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml'

In [446]:
#checking the data 
import pandas as pd

In [447]:
proj_link = 'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'
os.chdir(proj_link)



In [448]:
data = pd.read_csv("artifacts/data_ingestion/bank.csv")
print("The data shape is ", data.shape)
data.head()



The data shape is  (4521, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [449]:
#data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [450]:
# descriptive analysis
data.describe()


,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [451]:
# checking null values
data.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [452]:
#prepare the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_dir: Path
    STATUS_FILE: str
    all_schema: dict
    

In [453]:

from clientClassifier.constants import *
from clientClassifier.utils.common import read_yaml, create_directories

In [454]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_dir = config.unzip_dir,
            all_schema = schema
            
                
            )
        
        return data_validation_config

In [455]:
#creating components
import os
from clientClassifier import logger

In [456]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            
            data= pd.read_csv(self.config.unzip_dir)
            all_columns = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_columns:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as  f:
                        f.write(f"Column {col} not in schema")
                        
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as  f:
                        f.write(f"Validation status: {validation_status}")
                        
            return validation_status
        
        except Exception as e:
            logger.exception(e)
            raise e  

        
              
            
            
      

In [ ]:
#define the pipeline
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    
    data_validation = DataValidation(config=data_validation_config)
    
    status = data_validation.validate_all_columns()
    logger.info(f"All columns validation status: {status}")
    
    
except Exception as e:
    logger.exception(e)
    raise e


[2025-04-05 16:11:44,821: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 16:11:44,823: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 16:11:44,828: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-05 16:11:44,830: INFO: common: created directory at: artifacts]
[2025-04-05 16:11:44,832: INFO: common: created directory at: artifacts/data_validation]
[2025-04-05 16:11:44,851: INFO: 759138983: All columns validation status: False]
